In [1]:
import os
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import Point, box
from shapely.strtree import STRtree
from shapely.geometry.base import BaseGeometry
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import logging

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set global font to Times New Roman
plt.rcParams["font.family"] = "Times New Roman"

# Constant definitions
PROJECTED_CRS = "EPSG:32651"  # UTM Zone 51N

# File paths
lst_dir = "shanghai_LST/LST"
landuse_shp = "shanghai-latest-free.shp/gis_osm_landuse_a_free_1.shp"
water_shp = "shanghai-latest-free.shp/gis_osm_water_a_free_1.shp"
places_shp = "shanghai-latest-free.shp/gis_osm_places_a_free_1.shp"
traffic_shp = "shanghai-latest-free.shp/gis_osm_traffic_a_free_1.shp"
pois_shp = "shanghai-latest-free.shp/gis_osm_pois_a_free_1.shp"
roads_shp = "shanghai-latest-free.shp/gis_osm_roads_free_1.shp"
buildings_shp = "shanghai-latest-free.shp/gis_osm_buildings_a_free_1.shp"
transport_shp = "shanghai-latest-free.shp/gis_osm_transport_a_free_1.shp"
boundary_path = "OSMB-a05454319b28f099ff3da3a49c5dd21e484d8b2d.geojson"
output_dir = "Results"
processed_dir = "processed_data"


In [3]:
# Ensure output directory and processed data directory exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)

In [4]:
# Check file existence
def check_file_exists(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist")
    return file_path

def process_and_save_gdf(gdf, save_path, boundary_gdf, driver="GeoJSON"):
    """Process GeoDataFrame, convert to EPSG:4326 and clip to boundary, use GeoJSON to avoid Shapefile limitations"""
    logger.info(f"Original CRS: {gdf.crs}")
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs(epsg=4326)
    gdf = gdf[gdf.intersects(boundary_gdf.unary_union)]

    # Rename column names to fit Shapefile limitations
    if driver == "ESRI Shapefile":
        gdf.columns = [col[:10] if len(col) > 10 else col for col in gdf.columns]

    try:
        gdf.to_file(save_path, driver=driver)
    except Exception as e:
        logger.error(f"Failed to save {save_path}: {e}")
        raise
    return gdf

# Define function to load or process GeoDataFrame
def load_or_process_gdf(raw_gdf, save_path, boundary_gdf, driver="GeoJSON"):
    """Load processed GeoDataFrame or process and save new data"""
    if os.path.exists(save_path):
        try:
            return gpd.read_file(save_path)
        except Exception as e:
            logger.error(f"Failed to load {save_path}: {e}")
    logger.info(f"Processing and saving {save_path}...")
    return process_and_save_gdf(raw_gdf, save_path, boundary_gdf, driver)

In [5]:
# Load Shanghai boundary
boundary_save_path = os.path.join(processed_dir, "shanghai_boundary.geojson")
if os.path.exists(boundary_save_path):
    gdf_shanghai = gpd.read_file(boundary_save_path)
else:
    gdf_shanghai = gpd.read_file(check_file_exists(boundary_path))
    gdf_shanghai.to_file(boundary_save_path, driver="GeoJSON")

# Load various data
gdf_landuse = gpd.read_file(check_file_exists(landuse_shp))
gdf_water = gpd.read_file(check_file_exists(water_shp))
gdf_places = gpd.read_file(check_file_exists(places_shp))
gdf_traffic = gpd.read_file(check_file_exists(traffic_shp))
gdf_pois = gpd.read_file(check_file_exists(pois_shp))
gdf_roads = gpd.read_file(check_file_exists(roads_shp))
gdf_buildings = gpd.read_file(check_file_exists(buildings_shp))
gdf_transport = gpd.read_file(check_file_exists(transport_shp))

In [6]:
# Print data information
logger.info(f"Landuse fclass: {gdf_landuse['fclass'].unique()}")
logger.info(f"Buildings type: {gdf_buildings['type'].unique()}")
logger.info(f"Landuse GDF: {gdf_landuse.shape}")
logger.info(f"Roads GDF: {gdf_roads.shape}")
logger.info(f"Buildings GDF: {gdf_buildings.shape}")
logger.info(f"Transport GDF: {gdf_transport.shape}")

# Filter relevant data
gdf_industrial = gdf_landuse[gdf_landuse['fclass'].isin(['industrial'])].copy()
gdf_green = gdf_landuse[gdf_landuse['fclass'].isin(['park', 'forest', 'grass'])].copy()
gdf_water = gdf_water[gdf_water['fclass'] == 'water'].copy()
factory_power_types = ['industrial', 'factory']
gdf_factories = gdf_buildings[gdf_buildings['type'].isin(factory_power_types)].copy()
energy_related_types = ['industrial', 'depot', 'storage_tank']
gdf_power_plants = gdf_buildings[gdf_buildings['type'].isin(energy_related_types)].copy()

# Validate filtering results
logger.info(f"Industrial GDF: {gdf_industrial.shape}, Bounds: {gdf_industrial.total_bounds if not gdf_industrial.empty else 'Empty'}")
logger.info(f"Factories GDF: {gdf_factories.shape}, Bounds: {gdf_factories.total_bounds if not gdf_factories.empty else 'Empty'}")
logger.info(f"Power Plants GDF: {gdf_power_plants.shape}, Bounds: {gdf_power_plants.total_bounds if not gdf_power_plants.empty else 'Empty'}")
logger.info(f"Transport GDF: {gdf_transport.shape}, Bounds: {gdf_transport.total_bounds if not gdf_transport.empty else 'Empty'}")

# Process and save GeoDataFrame
gdf_list = [
    ("industrial_areas.geojson", gdf_industrial),
    ("green_areas.geojson", gdf_green),
    ("water_bodies.geojson", gdf_water),
    ("factories.geojson", gdf_factories),
    ("power_plants.geojson", gdf_power_plants),
    ("buildings.geojson", gdf_buildings),
    ("roads.geojson", gdf_roads),
    ("transport.geojson", gdf_transport)
]

for name, raw_gdf in gdf_list:
    save_path = os.path.join(processed_dir, name)
    globals()[f"gdf_{name.split('.')[0]}"] = load_or_process_gdf(raw_gdf, save_path, gdf_shanghai, driver="GeoJSON")

INFO:__main__:Landuse fclass: ['residential' 'park' 'retail' 'commercial' 'forest' 'recreation_ground'
 'grass' 'industrial' 'scrub' 'meadow' 'military' 'orchard' 'farmland'
 'cemetery' 'nature_reserve' 'farmyard' 'allotments' 'quarry' 'vineyard'
 'heath']
INFO:__main__:Buildings type: ['commercial' 'tower' None 'skyscraper' 'church' 'hall' 'train_station'
 'retail' 'pagoda' 'transportation' 'storage_tank' 'university'
 'industrial' 'stadium' 'apartments' 'fire_station' 'school' 'office'
 'roof' 'grandstand' 'residential' 'public' 'college' 'hotel'
 'electricity' 'hospital' 'house' 'supermarket' 'dormitory' 'garages'
 'detached' 'greenhouse' 'tent' 'no' 'manufacture' 'construction'
 'bus_station' 'terrace' 'kindergarten' 'hut' 'theatre' 'garage' 'hangar'
 'warehouse' 'shed' 'boathouse' 'service' 'conservatory' 'yes;roof'
 'sports' 'civic' 'temple' 'parking' 'depot' 'gymnasium' 'farm'
 'static_caravan' 'ruins' 'station' 'museum' 'gatehouse' 'government'
 'gallery' 'boat' 'police' 'yes;c

In [7]:
# Read and clip LST raster
def read_raster(file_path, boundary):
    """Read and clip raster data, return data, metadata, and transform"""
    with rasterio.open(file_path) as src:
        out_image, out_transform = mask(src, [boundary], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({'height': out_image.shape[1], 'width': out_image.shape[2], 'transform': out_transform})
        out_image = out_image.astype(np.float32)
        out_image[out_image == src.nodata] = np.nan
    return out_image[0], out_meta, src.meta['transform']

# Extract LST data for specific year and month
def get_lst_data(year, month):
    """Extract LST data for the specified year and month"""
    year_dir = os.path.join(lst_dir, str(year))
    if not os.path.exists(year_dir):
        logger.warning(f"{year_dir} does not exist")
        return None, None, None
    for filename in os.listdir(year_dir):
        if filename.endswith(".tif") and f"Day{year}-{month:02d}" in filename:
            file_path = os.path.join(year_dir, filename)
            return read_raster(file_path, gdf_shanghai.unary_union)
    logger.warning(f"No LST data found for {year}-{month:02d}")
    return None, None, None

# Extract the latest LST data
latest_year, latest_month = 2024, 12
latest_lst, latest_meta, transform = get_lst_data(latest_year, latest_month)
if latest_lst is None:
    raise ValueError("Unable to load LST data, please check file path and naming")

# Print LST information
height, width = latest_lst.shape
logger.info(f"LST Shape: {latest_lst.shape}")
logger.info(f"LST Valid Pixels: {np.sum(~np.isnan(latest_lst))}")
pixel_size = abs(transform[0])  # Pixel size (meters)
manual_pixel_size = (122.108 - 120.850) * 111000 * math.cos(math.radians(31.284)) / 141
logger.info(f"Calculated pixel size: {pixel_size} meters, Manual calculated pixel size: {manual_pixel_size} meters")
pixel_size = manual_pixel_size  # Use manual value meters/pixel

C:\Users\user\AppData\Local\Temp\ipykernel_36172\1976959811.py:22: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  return read_raster(file_path, gdf_shanghai.unary_union)
INFO:__main__:LST Shape: (132, 141)
INFO:__main__:LST Valid Pixels: 8422
INFO:__main__:Calculated pixel size: 0.00898315284119522 meters, Manual calculated pixel size: 846.3487641583322 meters


In [10]:
# Dynamically set buffer sizes
BUFFER_5KM = int(5000 / pixel_size)  # 5km buffer in pixels
BUFFER_10KM = int(10000 / pixel_size)  # 10km buffer in pixels
logger.info(f"Adjusted BUFFER_5KM: {BUFFER_5KM} pixels, BUFFER_10KM: {BUFFER_10KM} pixels")

# Extract longitude and latitude grid
x = np.array([transform[2] + transform[0] * j for j in range(width)])
y = np.array([transform[5] + transform[4] * i for i in range(height)])

# Feature extraction function
def extract_features(lst, meta, industrial_gdf, green_gdf, water_gdf, factories_gdf, power_plants_gdf, transport_gdf,
                     roads_gdf, buildings_gdf, lon, lat):
    """
    Extract LST and geographic features
    """
    df_save_path = os.path.join(processed_dir, "latest_df.csv")
    if os.path.exists(df_save_path):
        return pd.read_csv(df_save_path)
    
        # Convert to projected coordinate system and validate
    industrial_gdf_projected = industrial_gdf.to_crs(PROJECTED_CRS)
    green_gdf_projected = green_gdf.to_crs(PROJECTED_CRS)
    water_gdf_projected = water_gdf.to_crs(PROJECTED_CRS)
    factories_gdf_projected = factories_gdf.to_crs(PROJECTED_CRS)
    power_plants_gdf_projected = power_plants_gdf.to_crs(PROJECTED_CRS)
    transport_gdf_projected = transport_gdf.to_crs(PROJECTED_CRS)
    roads_gdf_projected = roads_gdf.to_crs(PROJECTED_CRS)
    buildings_gdf_projected = buildings_gdf.to_crs(PROJECTED_CRS)

    # Create STRtree spatial index
    industrial_tree = STRtree(industrial_gdf_projected.geometry) if not industrial_gdf_projected.empty else None
    green_tree = STRtree(green_gdf_projected.geometry) if not green_gdf_projected.empty else None
    water_tree = STRtree(water_gdf_projected.geometry) if not water_gdf_projected.empty else None
    factories_tree = STRtree(factories_gdf_projected.geometry) if not factories_gdf_projected.empty else None
    power_plants_tree = STRtree(power_plants_gdf_projected.geometry) if not power_plants_gdf_projected.empty else None
    transport_tree = STRtree(transport_gdf_projected.geometry) if not transport_gdf_projected.empty else None
    roads_tree = STRtree(roads_gdf_projected.geometry) if not roads_gdf_projected.empty else None
    buildings_tree = STRtree(buildings_gdf_projected.geometry) if not buildings_gdf_projected.empty else None

    # Extract longitude, latitude, and LST data for the entire grid
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    lst_flat = lst.ravel()

    # Create point geometries
    points = gpd.GeoSeries(
        [Point(lon_val, lat_val) for lon_val, lat_val in zip(lon_grid.ravel(), lat_grid.ravel())],
        crs="EPSG:4326"
    )
    points_projected = points.to_crs(PROJECTED_CRS)
    logger.info(f"Sample Point: {points_projected[0].wkt}")

    # Initialize feature arrays
    n_points = len(points)
    dist_to_industrial = np.full(n_points, np.nan)
    is_industrial = np.zeros(n_points, dtype=int)
    factory_counts = np.zeros(n_points)
    power_plant_flag = np.zeros(n_points, dtype=int)
    transport_hub_flag = np.zeros(n_points, dtype=int)
    road_counts = np.zeros(n_points)
    building_counts = np.zeros(n_points)
    dist_to_green = np.full(n_points, np.nan)
    dist_to_water = np.full(n_points, np.nan)

    # Buffer sizes (meters)
    buffer_1km_m = 5000  # 5km buffer (meters)
    buffer_5km_m = 10000  # 10km buffer (meters)

    # Calculate features
    for idx, p in enumerate(points_projected):
        # Industrial-related features
        if industrial_tree:
            # Query returns indices instead of geometry objects
            query_indices = industrial_tree.query(p.buffer(buffer_1km_m))
            if len(query_indices) > 0:
                # Retrieve actual geometry objects using indices
                query_geoms = industrial_gdf_projected.geometry.iloc[query_indices]
                logger.info(
                    f"Industrial Query for point {p.wkt}: {len(query_indices)} geoms found, Buffer size: {buffer_1km_m} m")
                is_industrial[idx] = 1 if any(p.intersects(geom) for geom in query_geoms) else 0

                # Find the nearest industrial area and calculate distance
                nearest_idx = industrial_tree.nearest(p)
                if nearest_idx is not None:  # Ensure a nearest object is found
                    nearest_geom = industrial_gdf_projected.geometry.iloc[nearest_idx]
                    dist_to_industrial[idx] = p.distance(nearest_geom) / 1000  # Convert to kilometers

        # Factory density
        if factories_tree:
            nearby_indices = factories_tree.query(p.buffer(buffer_1km_m))
            factory_counts[idx] = len(nearby_indices)
            logger.info(f"Factories Query: {len(nearby_indices)} geoms found")

        # Power plants and transport hubs
        if power_plants_tree:
            query_indices = power_plants_tree.query(p.buffer(buffer_5km_m))
            logger.info(f"Power Plants Query: {len(query_indices)} geoms found")
            if len(query_indices) > 0:
                # Retrieve all queried geometry objects
                query_geoms = power_plants_gdf_projected.geometry.iloc[query_indices]
                # Calculate minimum distance
                distances = [p.distance(geom) for geom in query_geoms]
                if distances:
                    nearest_dist = min(distances)
                    power_plant_flag[idx] = 1 if nearest_dist < buffer_5km_m else 0

        if transport_tree:
            query_indices = transport_tree.query(p.buffer(buffer_5km_m))
            logger.info(f"Transport Query: {len(query_indices)} geoms found")
            if len(query_indices) > 0:
                # Retrieve all queried geometry objects
                query_geoms = transport_gdf_projected.geometry.iloc[query_indices]
                # Calculate minimum distance
                distances = [p.distance(geom) for geom in query_geoms]
                if distances:
                    nearest_dist = min(distances)
                    transport_hub_flag[idx] = 1 if nearest_dist < buffer_5km_m else 0

        # Roads and buildings
        if roads_tree:
            nearby_indices = roads_tree.query(p.buffer(buffer_1km_m))
            road_counts[idx] = len(nearby_indices)

        if buildings_tree:
            nearby_indices = buildings_tree.query(p.buffer(buffer_1km_m))
            building_counts[idx] = len(nearby_indices)

        # Green areas and water bodies
        if green_tree:
            nearest_idx = green_tree.nearest(p)
            if nearest_idx is not None:
                nearest_geom = green_gdf_projected.geometry.iloc[nearest_idx]
                dist_to_green[idx] = p.distance(nearest_geom) / 1000  # Convert to kilometers

        if water_tree:
            nearest_idx = water_tree.nearest(p)
            if nearest_idx is not None:
                nearest_geom = water_gdf_projected.geometry.iloc[nearest_idx]
                dist_to_water[idx] = p.distance(nearest_geom) / 1000  # Convert to kilometers

    # Calculate density and impervious surface ratio
    buffer_areas = points_projected.buffer(buffer_1km_m).area
    factory_density = factory_counts / (buffer_areas / 1e6)  # Factories per square kilometer
    road_density = road_counts / (buffer_areas / 1e6)  # Roads per square kilometer
    impervious_ratio = (road_counts + building_counts) / (buffer_areas / 1e6 * 1000)  # Impervious surface ratio

    # Create DataFrame
    df = pd.DataFrame({
        'lst': lst_flat,
        'lon': lon_grid.ravel(),
        'lat': lat_grid.ravel(),
        'dist_to_industrial': dist_to_industrial,
        'is_industrial': is_industrial,
        'factory_density': factory_density,
        'power_plant_flag': power_plant_flag,
        'transport_hub_flag': transport_hub_flag,
        'road_density': road_density,
        'impervious_ratio': impervious_ratio,
        'dist_to_green': dist_to_green,
        'dist_to_water': dist_to_water
    }).dropna(subset=['lst'])

    try:
        df.to_csv(df_save_path, index=False)
        logger.info(f"Successfully saved DataFrame to {df_save_path}")
    except Exception as e:
        logger.error(f"Failed to save DataFrame: {e}")
        raise

    return df

INFO:__main__:Adjusted BUFFER_5KM: 5 pixels, BUFFER_10KM: 11 pixels


In [11]:
# Extract features
latest_df = extract_features(latest_lst, latest_meta, gdf_industrial, gdf_green, gdf_water, gdf_factories,
                             gdf_power_plants, gdf_transport, gdf_roads, gdf_buildings, x, y)

In [12]:
# Calculate industrial land density
industrial_density = np.zeros_like(latest_lst)
for idx, row in latest_df.iterrows():
    if row['is_industrial'] == 1:
        i = np.searchsorted(y, row['lat'], side='right') - 1
        j = np.searchsorted(x, row['lon'], side='right') - 1
        if 0 <= i < height and 0 <= j < width:
            industrial_density[i, j] += 1

In [13]:
# Getis-Ord Gi* statistic
def calculate_getis_ord_gi_star(data, lat, lon, dist_threshold=0.02):
    """Calculate Getis-Ord Gi* statistic to identify hotspots"""
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    points = np.column_stack((lat_grid.ravel(), lon_grid.ravel()))
    data_flat = data.ravel()
    valid_mask = ~np.isnan(data_flat)
    data_flat, points = data_flat[valid_mask], points[valid_mask]
    n = len(data_flat)
    mean_x, std_x = np.mean(data_flat), np.std(data_flat)
    Gi_star = np.full(n, np.nan)
    for i, point in enumerate(points):
        distances = np.sqrt(((points - point) ** 2).sum(axis=1))
        neighbors = distances <= dist_threshold
        local_sum = np.sum(data_flat[neighbors])
        weight_sum = np.sum(neighbors)
        weight_sq_sum = np.sum(neighbors ** 2)
        numerator = local_sum - mean_x * weight_sum
        denominator = std_x * np.sqrt((n * weight_sq_sum - weight_sum ** 2) / (n - 1)) if n > 1 else 1
        Gi_star[i] = numerator / denominator if denominator != 0 else 0
    Gi_star_grid = np.full(lat_grid.shape, np.nan)
    Gi_star_grid.ravel()[valid_mask] = Gi_star
    return Gi_star_grid

In [14]:
# Calculate UHI and industrial hotspots
lst_gi_star = calculate_getis_ord_gi_star(latest_lst, y, x, dist_threshold=0.02)
industrial_gi_star = calculate_getis_ord_gi_star(industrial_density, y, x, dist_threshold=0.02)

# Determine LST data range
lst_bounds = box(min(x.flatten()), min(y.flatten()), max(x.flatten()), max(y.flatten()))
logger.info(f"LST data range: {lst_bounds.bounds}")

# Convert LST range to GeoDataFrame for spatial filtering
lst_bounds_gdf = gpd.GeoDataFrame(geometry=[lst_bounds], crs="EPSG:4326")

# Ensure gdf_shanghai's CRS matches LST data (EPSG:4326)
if gdf_shanghai.crs != "EPSG:4326":
    gdf_shanghai = gdf_shanghai.to_crs("EPSG:4326")

# Filter gdf_shanghai for parts intersecting LST data range
gdf_shanghai_subset = gdf_shanghai[gdf_shanghai.intersects(lst_bounds)]
logger.info(f"Number of filtered Shanghai boundary parts: {len(gdf_shanghai_subset)}")

# Log warning if gdf_shanghai_subset is empty
if gdf_shanghai_subset.empty:
    logger.warning("No Shanghai boundary parts intersecting LST data range found!")

# Plot UHI and industrial hotspots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
im1 = ax1.pcolormesh(x, y, lst_gi_star, cmap="RdBu", shading="auto")
ax1.set_title("UHI Hotspot (Gi*)", fontsize=14)
plt.colorbar(im1, ax=ax1, label="Gi*")
if not gdf_shanghai_subset.empty:
    gdf_shanghai_subset.boundary.plot(ax=ax1, color="black", linewidth=1)

im2 = ax2.pcolormesh(x, y, industrial_gi_star, cmap="YlOrRd", shading="auto")
ax2.set_title("Industrial Hotspot (Gi*)", fontsize=14)
plt.colorbar(im2, ax=ax2, label="Gi*")
if not gdf_shanghai_subset.empty:
    gdf_shanghai_subset.boundary.plot(ax=ax2, color="black", linewidth=1)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "uhi_industrial_hotspots.png"), dpi=300)
plt.close()

# Visualize hotspot areas and industrial zones
plt.figure(figsize=(10, 8))
latest_df['is_hotspot'] = latest_df['lst'] > latest_df['lst'].quantile(0.70)
plt.scatter(latest_df['lon'], latest_df['lat'], c=latest_df['is_hotspot'], cmap='Reds', s=1)
if not gdf_industrial.empty:
    # Filter gdf_industrial for parts intersecting LST range
    gdf_industrial_subset = gdf_industrial[gdf_industrial.intersects(lst_bounds)]
    if not gdf_industrial_subset.empty:
        gdf_industrial_subset.boundary.plot(ax=plt.gca(), color='blue', linewidth=1)
    else:
        logger.warning("No industrial zones intersecting LST data range found!")
plt.title("Industrial & UHI")
plt.savefig(os.path.join(output_dir, 'hotspot_industrial.png'), dpi=300)
plt.close()

INFO:__main__:LST data range: (120.85035517259927, 30.695433258364066, 122.1079965703666, 31.87222628056064)
INFO:__main__:Number of filtered Shanghai boundary parts: 1


In [15]:
# Industrial feature analysis within hotspot areas
hotspot_df = latest_df[latest_df['is_hotspot']]
logger.info(f"Number of hotspot pixels: {latest_df['is_hotspot'].sum()}")
print(f"Industrial land proportion in hotspot areas: {hotspot_df['is_industrial'].mean():.2%}")
print(f"Factory density in hotspot areas: {hotspot_df['factory_density'].mean():.4f} factories/km²")
print(f"Coal power plant proportion in hotspot areas: {hotspot_df['power_plant_flag'].mean():.2%}")
print(f"Transport hub proportion in hotspot areas: {hotspot_df['transport_hub_flag'].mean():.2%}")
print(f"Impervious surface ratio in hotspot areas: {hotspot_df['impervious_ratio'].mean():.2%}")
print(f"Distance to green areas in hotspot areas: {hotspot_df['dist_to_green'].mean():.4f} km")
print(f"Distance to water bodies in hotspot areas: {hotspot_df['dist_to_water'].mean():.4f} km")

INFO:__main__:Number of hotspot pixels: 2527


Industrial land proportion in hotspot areas: 11.40%
Factory density in hotspot areas: 1.6298 factories/km²
Coal power plant proportion in hotspot areas: 97.15%
Transport hub proportion in hotspot areas: 92.40%
Impervious surface ratio in hotspot areas: 7.36%
Distance to green areas in hotspot areas: 1.0559 km
Distance to water bodies in hotspot areas: 1.1236 km


In [17]:
# Machine learning analysis
# Define features and target for classification
features = ['dist_to_industrial', 'is_industrial', 'factory_density', 'power_plant_flag',
            'transport_hub_flag', 'road_density', 'impervious_ratio', 'dist_to_green', 'dist_to_water']
X = latest_df[features]
y = latest_df['is_hotspot'].astype(int)  # Binary classification: 1 (hotspot), 0 (non-hotspot)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
logger.info(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")

INFO:__main__:Training set size: 6737, Test set size: 1685


In [18]:
# Initialize models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')

# Train Random Forest
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probabilities for RMSE and R²

# Train XGBoost
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probabilities for RMSE and R²

In [19]:
# Evaluate models - Classification metrics
def evaluate_classification_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"{model_name} - Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
    return accuracy, precision, recall, f1

# Evaluate models - Regression-like metrics (using probabilities)
def evaluate_regression_metrics(y_true, y_pred_proba, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred_proba))
    r2 = r2_score(y_true, y_pred_proba)
    print(f"{model_name} - RMSE (using probabilities): {rmse:.2f}, R²: {r2:.2f}")
    return rmse, r2

# Evaluate Random Forest
rf_metrics = evaluate_classification_model(y_test, rf_pred, "Random Forest Classifier")
rf_rmse, rf_r2 = evaluate_regression_metrics(y_test, rf_pred_proba, "Random Forest Classifier")

# Evaluate XGBoost
xgb_metrics = evaluate_classification_model(y_test, xgb_pred, "XGBoost Classifier")
xgb_rmse, xgb_r2 = evaluate_regression_metrics(y_test, xgb_pred_proba, "XGBoost Classifier")

# Confusion Matrix Visualization
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Non-Hotspot', 'Hotspot'], yticklabels=['Non-Hotspot', 'Hotspot'])
    plt.title(f'Confusion Matrix - {model_name}', fontsize=14)
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.savefig(os.path.join(output_dir, f'confusion_matrix_{model_name.lower().replace(" ", "_")}.png'), dpi=300)
    plt.close()

# Plot confusion matrices
plot_confusion_matrix(y_test, rf_pred, "Random Forest Classifier")
plot_confusion_matrix(y_test, xgb_pred, "XGBoost Classifier")

# Feature Importance for Random Forest
rf_importance = pd.DataFrame({'feature': features, 'importance': rf_model.feature_importances_}).sort_values('importance', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=rf_importance)
plt.title('Feature Importance - Random Forest Classifier', fontsize=14)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.savefig(os.path.join(output_dir, 'feature_importance_rf_classifier.png'), dpi=300)
plt.close()

# Feature Importance for XGBoost
xgb_importance = pd.DataFrame({'feature': features, 'importance': xgb_model.feature_importances_}).sort_values('importance', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=xgb_importance)
plt.title('Feature Importance - XGBoost Classifier', fontsize=14)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.savefig(os.path.join(output_dir, 'feature_importance_xgb_classifier.png'), dpi=300)
plt.close()

Random Forest Classifier - Accuracy: 0.79, Precision: 0.71, Recall: 0.51, F1-Score: 0.60
Random Forest Classifier - RMSE (using probabilities): 0.38, R²: 0.32
XGBoost Classifier - Accuracy: 0.78, Precision: 0.67, Recall: 0.54, F1-Score: 0.59
XGBoost Classifier - RMSE (using probabilities): 0.38, R²: 0.31
